In [1]:
# FABRIC Configuration
SLICE_NAME = "bgp_sec_test"
SITE_NAME = "WASH"
MEAS_ADD = False

# Folded-Clos Configuration
PORTS_PER_DEVICE = 4
NUMBER_OF_TIERS = 2
NETWORK_NODE_PREFIXES = "T,S,L"
COMPUTE_NODE_PREFIXES = "C"
SEC_NODE_PREFIX = "H"
SINGLE_COMPUTE_SUBNET = False
SOUTHBOUND_PORT_DENSITY = {1:1}
ADD_SEC_NODE = True
BGP_SCRIPTS_LOCATION = "/home/fabric/work/NShenoy/FABRIC-Automation-main/remote_scripts/bgp_scripts"
TEMPLATE_LOCATION = "/home/fabric/work/NShenoy/FABRIC-Automation-main/remote_scripts/frr_templates/frr_conf_bgp.mako"

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
    fablib.show_config()
    print (fablib.get_image_names())


except Exception as e:
    print(f"Exception: {e}")

ModuleNotFoundError: No module named 'fabrictestbed_extensions'

## <span style="color: #de4815"><b>Build a Graph of the Folded-Clos Topology</b></span>

A custom library, ClosGenerator, is used to build a graph-representation of a folded-Clos topology. BGP-4 configuration is added as attributes to the nodes as the graph is being constructed.

In [ ]:
from infra_scripts.bgp.ClosGenerator import *

topology = BGPDCNConfig(PORTS_PER_DEVICE,
                        NUMBER_OF_TIERS,
                        southboundPortsConfig=SOUTHBOUND_PORT_DENSITY,
                        singleComputeSubnet=SINGLE_COMPUTE_SUBNET,
                        addSecurityNode=ADD_SEC_NODE)
topology.buildGraph()
logFile = topology.jsonGraphInfo()

print("BGP configuration complete\n")
print(f"Folded-Clos topology details (Not considering port density changes and security node additions):\n{topology.getClosStats()}")

BGP configuration complete

Folded-Clos topology details (Not considering port density changes and security node additions):
Number of ToF Nodes: 2
Number of physical servers: 8
Number of networking nodes: 6
Number of leaves: 4
Number of Pods: 1



## <span style="color: #de4815"><b>Prepare the BGP Configuration Template</b></span>

This book uses the Mako template engine to populate BGP-related information into the default FRR configuration file (frr.conf). The per-node BGP configuration is contained in the graph structure built in the prior section.

In [ ]:
from mako.template import Template

try:
    bgpTemplate = Template(filename=TEMPLATE_LOCATION)
    print("FRR-BGP configuration template loaded.")

except Exception as e:
    print(f"Exception: {e}")

FRR-BGP configuration template loaded.


## <span style="color: #de4815"><b>Parse the folded-Clos Configuration and Create the Slice</b></span>

In [ ]:
import os

# CONFIGURATION FOR BGP-SPEAKING LEAF AND SPINE NODES
def addBGPConfiguration(node, nodeInfo, topology, bgpTemplate):
    '''
    Prepare a node for the FRR BGP-4 implementation to be installed.
    '''

    # Store information about BGP-speaking neighbors to configure neighborship
    neighboringNodes = []

    # Find the node's BGP-speaking neighbors and determine their ASN as well as their IPv4 address used on the subnet shared by the nodes.
    for neighbor, addr in topology.getNodeAttribute(node, 'ipv4').items():
        if(topology.isNetworkNode(neighbor)):
            neighboringNodes.append({'asn':topology.getNodeAttribute(neighbor, 'ASN'), 'ip':topology.getNodeAttribute(neighbor, 'ipv4', node)})

    # In addition to storing neighbor information, store any compute subnets that the node must advertise to neighbors (leaf's only).
    nodeTemplate = {'neighbors':neighboringNodes, 'bgp_asn': topology.getNodeAttribute(node, 'ASN'), 'networks': topology.getNodeAttribute(node, 'advertise')}

    # Process the stored information and render a custom frr.conf.
    nodeBGPData = bgpTemplate.render(**nodeTemplate)

    # Add FABRIC post-boot tasks to get the node ready for FRR installation
    nodeInfo.add_post_boot_upload_directory(BGP_SCRIPTS_LOCATION,'.')
    nodeInfo.add_post_boot_execute(f'sudo echo -e "{nodeBGPData}" > bgp_scripts/frr.conf')
    nodeInfo.add_post_boot_execute('sudo chmod +x /home/rocky/bgp_scripts/*.sh')

    return

# CONFIGURATION FOR NON-BGP-SPEAKING COMPUTE NODES
def addComputeConfiguration(nodeInfo):
    '''
    Prepare a node for traffic testing.
    '''
    nodeInfo.add_post_boot_upload_directory(BGP_SCRIPTS_LOCATION,'.')
    nodeInfo.add_post_boot_execute('sudo chmod +x /home/rocky/bgp_scripts/*.sh') # added sudo to the front of both of them and added all *.sh to execute

    return

# Create the slice
slice = fablib.new_slice(name=SLICE_NAME)

addedNodes = {} # Visited nodes structure, format = name : nodeInfo

# Add slice-specific information to the log file
logFile.update({"name": SLICE_NAME, "site": SITE_NAME, "meas": MEAS_ADD})

# Iterate over each network in the topology and configure each interface connected to the network, and the network itself.
for networkInfo in topology.iterNetwork(fabricFormating=True):
    networkIntfs = [] # Interfaces to be added to the network.
    network = networkInfo[0] # A tuple containing the nodes on the network.
    networkName = networkInfo[1] # The FABRIC network name.

    print(f"Configuring network: {network}")

    # For each node in a given IPv4 subnet within the folded-Clos topology.
    for node in network:
        # If the node has not yet been visited, provide it with the appropriate configuration.
        if(node not in addedNodes):
            # Add the node to the FABRIC slice.
            if(node.startswith(SEC_NODE_PREFIX)):
                nodeInfo = slice.add_node(name=node, cores=4, ram=4, disk=80, image='default_kali', site=SITE_NAME)
            else:
                nodeInfo = slice.add_node(name=node, cores=1, ram=4, image='default_rocky_8', site=SITE_NAME)

            # If the node is a non-compute node, it needs FRR-BGP configuration instructions.
            if(topology.isNetworkNode(node)):
                addBGPConfiguration(node, nodeInfo, topology, bgpTemplate)
            else:
                addComputeConfiguration(nodeInfo)

            addedNodes[node] = nodeInfo
            print(f"\tAdded node {node} to the slice.")

        else:
            print(f"\tAlready added node {node} to the slice.")

        # Create a name for the node's interface connected to this network and add it to the FABRIC slice.
        intfName = topology.generateFabricIntfName(node, network)
        netIntf = addedNodes[node].add_component(model='NIC_Basic', name=intfName).get_interfaces()[0]
        networkIntfs.append(netIntf)

    # Add the network to the FABRIC slice.
    slice.add_l2network(name=networkName, interfaces=networkIntfs, type="L2Bridge")
    print(f"\tAdded network {network}")

Configuring network: ('T-1', 'L-1')
	Added node T-1 to the slice.
	Added node L-1 to the slice.
	Added network ('T-1', 'L-1')
Configuring network: ('T-1', 'H-1')
	Already added node T-1 to the slice.
	Added node H-1 to the slice.
	Added network ('T-1', 'H-1')
Configuring network: ('T-1', 'L-2')
	Already added node T-1 to the slice.
	Added node L-2 to the slice.
	Added network ('T-1', 'L-2')
Configuring network: ('T-1', 'L-3')
	Already added node T-1 to the slice.
	Added node L-3 to the slice.
	Added network ('T-1', 'L-3')
Configuring network: ('T-1', 'L-4')
	Already added node T-1 to the slice.
	Added node L-4 to the slice.
	Added network ('T-1', 'L-4')
Configuring network: ('L-1', 'T-2')
	Already added node L-1 to the slice.
	Added node T-2 to the slice.
	Added network ('L-1', 'T-2')
Configuring network: ('L-1', 'C-1-1')
	Already added node L-1 to the slice.
	Added node C-1-1 to the slice.
	Added network ('L-1', 'C-1-1')
Configuring network: ('L-2', 'T-2')
	Already added node L-2 to t

## <span style="color: #de4815"><b>Add a Measurement Node (Optional)</b></span>

In [ ]:
if(MEAS_ADD):
    import mflib
    print(f"MFLib version  {mflib.__version__} " )

    from mflib.mflib import MFLib

    # Add measurement node to topology using static method.
    MFLib.addMeasNode(slice, disk=100, image='docker_ubuntu_20', site=SITE_NAME)
    print("Measurement node added.")

else:
    print("No measurement node added.")

## <span style="color: #de4815"><b>Submit the Slice</b></span>

In [ ]:
%%time
import json

try:
    # Submit Slice Request
    print(f'Submitting the new slice, "{SLICE_NAME}"...')
    slice.submit()
    print(f'{SLICE_NAME} creation done.')

except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


Retry: 14, Time: 565 sec


ID,9bf97198-7417-47ab-94c9-290e328051a3
Name,bgp_sec_test
Lease Expiration (UTC),2024-09-06 13:57:41 +0000
Lease Start (UTC),2024-09-05 13:57:41 +0000
Project ID,787adfc9-d37e-42f2-8efe-8e32793e0bb8
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
cfd430c9-3470-43a5-b3c4-057cbc75510d,C-1-1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe3d:b271,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe3d:b271,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
413c7c93-31a1-4357-92f5-522038abd052,C-2-1,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe09:dd6e,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe09:dd6e,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
3c546d1b-5e93-4584-bd68-aa540c090893,C-3-1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fed3:20ad,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fed3:20ad,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
a4a150bd-acb7-453c-8dfb-da224af847e5,C-4-1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe5c:d850,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe5c:d850,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
f890aa23-451e-4c9b-a7cb-c43964a24222,H-1,4,4,100,default_kali,qcow2,wash-w3.fabric-testbed.net,WASH,kali,2001:400:a100:3020:f816:3eff:fe40:6c51,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config kali@2001:400:a100:3020:f816:3eff:fe40:6c51,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
70774450-0f23-4aab-b773-3744fa2cd1ec,L-1,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:febc:5c4a,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:febc:5c4a,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
346d6850-48e4-4b80-baf9-b781e835ad16,L-2,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe49:7b26,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe49:7b26,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
b81d7cd7-e924-495f-a463-0166221a2c62,L-3,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe63:4ea6,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe63:4ea6,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
b34e8bee-4086-4602-9a97-18a0ff1a0cde,L-4,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe06:8b03,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe06:8b03,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
7bef2db5-291f-4ac7-afef-cd9ce44c849b,T-1,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:feca:433a,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:feca:433a,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b6f255f4-708b-4cc7-a4f3-761297255cb0,core-T-1-L-1,L2,L2Bridge,WASH,None,None,Active,
d3871ffa-d2a4-46ea-9043-2aabd5f93a8e,core-T-1-L-2,L2,L2Bridge,WASH,None,None,Active,
7e38e6a4-ac33-49bf-a4ee-64f1030bd82b,core-T-1-L-3,L2,L2Bridge,WASH,None,None,Active,
ce1b4b61-e477-48e2-87ae-15c7dcefac08,core-T-1-L-4,L2,L2Bridge,WASH,None,None,Active,
9584c4d5-6dfe-429f-9bda-3e15b21b6114,core-T-2-L-1,L2,L2Bridge,WASH,None,None,Active,
1094dd8e-009e-49f3-8fa9-556bf6f36c7c,core-T-2-L-2,L2,L2Bridge,WASH,None,None,Active,
42b14fc5-d438-4b79-8439-2b11bcafbfb1,core-T-2-L-3,L2,L2Bridge,WASH,None,None,Active,
2622e694-7008-432d-96f1-961a4b8d54b3,core-T-2-L-4,L2,L2Bridge,WASH,None,None,Active,
fff372d5-ca20-4833-8006-4ab517d28f95,edge-L-1-C-1-1,L2,L2Bridge,WASH,None,None,Active,
1d2c01e3-b278-41de-930c-523017f805b1,edge-L-2-C-2-1,L2,L2Bridge,WASH,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
T-1-intf-L-2-p1,p1,T-1,core-T-1-L-2,100,config,,C2:4B:2B:AA:42:8E,eth5,eth5,None,6,HundredGigE0/0/0/5
T-1-intf-H-1-p1,p1,T-1,edge-T-1-H-1,100,config,,BE:D4:49:3E:8C:48,eth4,eth4,None,6,HundredGigE0/0/0/5
T-1-intf-L-4-p1,p1,T-1,core-T-1-L-4,100,config,,BE:86:04:21:C2:9A,eth3,eth3,None,6,HundredGigE0/0/0/5
T-1-intf-L-1-p1,p1,T-1,core-T-1-L-1,100,config,,BE:2E:EC:2C:43:BF,eth2,eth2,None,6,HundredGigE0/0/0/5
T-1-intf-L-3-p1,p1,T-1,core-T-1-L-3,100,config,,BE:0D:C0:9B:8C:2D,eth1,eth1,None,6,HundredGigE0/0/0/5
L-1-intf-C-1-1-p1,p1,L-1,edge-L-1-C-1-1,100,config,,C2:83:78:0A:2D:49,eth1,eth1,None,6,HundredGigE0/0/0/5
L-1-intf-T-1-p1,p1,L-1,core-T-1-L-1,100,config,,C6:C7:84:65:95:28,eth3,eth3,None,6,HundredGigE0/0/0/5
L-1-intf-T-2-p1,p1,L-1,core-T-2-L-1,100,config,,C6:A5:A5:3E:CB:A6,eth2,eth2,None,6,HundredGigE0/0/0/5
H-1-intf-T-1-p1,p1,H-1,edge-T-1-H-1,100,config,,4E:6B:86:D3:40:40,eth1,eth1,fe80::4c6b:86ff:fed3:4040,4,HundredGigE0/0/0/9
L-2-intf-C-2-1-p1,p1,L-2,edge-L-2-C-2-1,100,config,,CA:F8:BE:67:9B:9C,eth2,eth2,None,6,HundredGigE0/0/0/5



Time to print interfaces 639 seconds
bgp_sec_test creation done.
CPU times: user 1min 18s, sys: 4.07 s, total: 1min 22s
Wall time: 10min 52s


## <span style="color: #de4815"><b>Initalize the Measurement Framework (Optional)</b></span>

This step both initalizes and instrumentizes the Measurement framework to use Prometheus and Grafana. If ELK is desired, modifications to the cell need to be made.

In [ ]:
if(MEAS_ADD):
    mf = MFLib(SLICE_NAME) # Initalize
    instrumetize_results = mf.instrumentize( ["prometheus"] ) # Instrumentize

    # Grafana SSH Tunnel Command
    print(mf.grafana_tunnel)
    print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

## <span style="color: #de4815"><b>Provide Initial Configuration to Nodes</b></span>

If the node is a core (BGP-speaking) node, it will have FRR installed and the BGP daemon (bgpd) turned on as well as configured.

If the node is an edge (non-BGP-speaking) node, it will have the traffic generator code installed.

In [ ]:
SLICE_NAME = "bgp_sec_test"
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
    fablib.show_config()

except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,787adfc9-d37e-42f2-8efe-8e32793e0bb8
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,nxsvks_0000031803
Bastion Private Key File,/home/fabric/work/fabric_config/BastKey
Slice Private Key File,/home/fabric/work/fabric_config/SliceKey
Slice Public Key File,/home/fabric/work/fabric_config/SliceKey.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)

except Exception as e:
    print(f"Exception: {e}")

Slice name: bgp_sec_test
Slice and nodes were acquired successfully.


In [ ]:
# Commands to execute the bash scripts configuring the nodes
coreNodeConfig = "./bgp_scripts/init_bgp.sh"
edgeNodeConfig = "./bgp_scripts/init_compute.sh"

# Configure core (BGP-speaking) nodes
manager.executeCommandsParallel(coreNodeConfig, prefixList=NETWORK_NODE_PREFIXES)

# Configure edge (non-BGP-speaking) nodes
manager.executeCommandsParallel(edgeNodeConfig, prefixList=COMPUTE_NODE_PREFIXES)

## <span style="color: #de4815"><b>Add IPv4 Addressing to All Nodes</b></span>

This system utilizes the addressing provided by the ClosGenerator module:

* 192.168.0.0/16 is the compute supernet. All compute subnets are given a /24 subnet. Compute devices are given lower addresses (ex: .1) and the leaf node is given a high address (ex: .254)

* 172.16.0.0/12 is the core supernet. All core subnets are given a /24 subnet. Both devices are given lower addresses.

In [ ]:
SLICE_NAME = "bgp_sec_test" #provide slicename
slice = fablib.get_slice(name=SLICE_NAME)
#for node in slice.get_nodes(): # nirmala added this because the nodes did not have a route afetr we stopped the NM service
#    node.execute("sudo systemctl start NetworkManager.service")
slice.show()
slice.list_nodes()
slice.list_networks()

ID,9bf97198-7417-47ab-94c9-290e328051a3
Name,bgp_sec_test
Lease Expiration (UTC),2024-09-06 13:57:41 +0000
Lease Start (UTC),2024-09-05 13:57:41 +0000
Project ID,787adfc9-d37e-42f2-8efe-8e32793e0bb8
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
cfd430c9-3470-43a5-b3c4-057cbc75510d,C-1-1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe3d:b271,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe3d:b271,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
413c7c93-31a1-4357-92f5-522038abd052,C-2-1,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe09:dd6e,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe09:dd6e,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
3c546d1b-5e93-4584-bd68-aa540c090893,C-3-1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fed3:20ad,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fed3:20ad,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
a4a150bd-acb7-453c-8dfb-da224af847e5,C-4-1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe5c:d850,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe5c:d850,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
f890aa23-451e-4c9b-a7cb-c43964a24222,H-1,4,4,100,default_kali,qcow2,wash-w3.fabric-testbed.net,WASH,kali,2001:400:a100:3020:f816:3eff:fe40:6c51,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config kali@2001:400:a100:3020:f816:3eff:fe40:6c51,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
70774450-0f23-4aab-b773-3744fa2cd1ec,L-1,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:febc:5c4a,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:febc:5c4a,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
346d6850-48e4-4b80-baf9-b781e835ad16,L-2,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe49:7b26,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe49:7b26,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
b81d7cd7-e924-495f-a463-0166221a2c62,L-3,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe63:4ea6,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe63:4ea6,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
b34e8bee-4086-4602-9a97-18a0ff1a0cde,L-4,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe06:8b03,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe06:8b03,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey
7bef2db5-291f-4ac7-afef-cd9ce44c849b,T-1,1,4,10,default_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:feca:433a,Active,,ssh -i /home/fabric/work/fabric_config/SliceKey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:feca:433a,/home/fabric/work/fabric_config/SliceKey.pub,/home/fabric/work/fabric_config/SliceKey


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b6f255f4-708b-4cc7-a4f3-761297255cb0,core-T-1-L-1,L2,L2Bridge,WASH,None,None,Active,
d3871ffa-d2a4-46ea-9043-2aabd5f93a8e,core-T-1-L-2,L2,L2Bridge,WASH,None,None,Active,
7e38e6a4-ac33-49bf-a4ee-64f1030bd82b,core-T-1-L-3,L2,L2Bridge,WASH,None,None,Active,
ce1b4b61-e477-48e2-87ae-15c7dcefac08,core-T-1-L-4,L2,L2Bridge,WASH,None,None,Active,
9584c4d5-6dfe-429f-9bda-3e15b21b6114,core-T-2-L-1,L2,L2Bridge,WASH,None,None,Active,
1094dd8e-009e-49f3-8fa9-556bf6f36c7c,core-T-2-L-2,L2,L2Bridge,WASH,None,None,Active,
42b14fc5-d438-4b79-8439-2b11bcafbfb1,core-T-2-L-3,L2,L2Bridge,WASH,None,None,Active,
2622e694-7008-432d-96f1-961a4b8d54b3,core-T-2-L-4,L2,L2Bridge,WASH,None,None,Active,
fff372d5-ca20-4833-8006-4ab517d28f95,edge-L-1-C-1-1,L2,L2Bridge,WASH,None,None,Active,
1d2c01e3-b278-41de-930c-523017f805b1,edge-L-2-C-2-1,L2,L2Bridge,WASH,None,None,Active,


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b6f255f4-708b-4cc7-a4f3-761297255cb0,core-T-1-L-1,L2,L2Bridge,WASH,None,None,Active,
d3871ffa-d2a4-46ea-9043-2aabd5f93a8e,core-T-1-L-2,L2,L2Bridge,WASH,None,None,Active,
7e38e6a4-ac33-49bf-a4ee-64f1030bd82b,core-T-1-L-3,L2,L2Bridge,WASH,None,None,Active,
ce1b4b61-e477-48e2-87ae-15c7dcefac08,core-T-1-L-4,L2,L2Bridge,WASH,None,None,Active,
9584c4d5-6dfe-429f-9bda-3e15b21b6114,core-T-2-L-1,L2,L2Bridge,WASH,None,None,Active,
1094dd8e-009e-49f3-8fa9-556bf6f36c7c,core-T-2-L-2,L2,L2Bridge,WASH,None,None,Active,
42b14fc5-d438-4b79-8439-2b11bcafbfb1,core-T-2-L-3,L2,L2Bridge,WASH,None,None,Active,
2622e694-7008-432d-96f1-961a4b8d54b3,core-T-2-L-4,L2,L2Bridge,WASH,None,None,Active,
fff372d5-ca20-4833-8006-4ab517d28f95,edge-L-1-C-1-1,L2,L2Bridge,WASH,None,None,Active,
1d2c01e3-b278-41de-930c-523017f805b1,edge-L-2-C-2-1,L2,L2Bridge,WASH,None,None,Active,


In [ ]:
from infra_scripts.bgp.ClosGenerator import *
from ipaddress import ip_address, IPv4Address, IPv4Network
import re

COMPUTE_SUPERNET = "192.168.0.0/16"

# Iterate through every node in the topology
for node in topology.iterNodes():
    print(f"Configuring IPv4 addressing on node: {node}")

    # Pull IPv4 attribute data to configure FABRIC interfaces
    for neighbor, currentAddress in topology.getNodeAttribute(node, 'ipv4').items():
        # Access the interface from FABRIC.
        intfName = f"{node}-intf-{neighbor}-p1" # Naming is a bit strange, but is formatted in FABRIC as such.
        intf = slice.get_interface(intfName)

        # Convert the address and subnet into ipaddress objects for FABRIC processing.
        fabAddress = IPv4Address(currentAddress)
        fabSubnet = IPv4Network(f"{currentAddress}/24", strict=False)

        # Assign the address to the interface.
        intf.ip_addr_add(addr=fabAddress, subnet=fabSubnet)

    # For compute nodes, also add a compute supernet route with its attached leaf node as the next-hop.
    if(not topology.isNetworkNode(node)):
        IPGroup = re.search(r"192\.168\.([0-9]{1,3})\.[0-9]{1,3}", currentAddress) # Grab the third octet number.
        thirdOctet = IPGroup.group(1)
        nextHop = f"192.168.{thirdOctet}.254"

        # Add the route to the node
        intf.get_node().ip_route_add(subnet=IPv4Network(COMPUTE_SUPERNET), gateway=IPv4Address(nextHop))

        print(f"\tAdded route to {node}")

    print("\tConfiguration complete.")

Configuring IPv4 addressing on node: T-1
	Configuration complete.
Configuring IPv4 addressing on node: L-1
	Configuration complete.
Configuring IPv4 addressing on node: H-1
	Added route to H-1
	Configuration complete.
Configuring IPv4 addressing on node: L-2
	Configuration complete.
Configuring IPv4 addressing on node: L-3
	Configuration complete.
Configuring IPv4 addressing on node: L-4
	Configuration complete.
Configuring IPv4 addressing on node: T-2
	Configuration complete.
Configuring IPv4 addressing on node: C-1-1
	Added route to C-1-1
	Configuration complete.
Configuring IPv4 addressing on node: C-2-1
	Added route to C-2-1
	Configuration complete.
Configuring IPv4 addressing on node: C-3-1
	Added route to C-3-1
	Configuration complete.
Configuring IPv4 addressing on node: C-4-1
	Added route to C-4-1
	Configuration complete.


## <span style="color: #de4815"><b>Log Topology Information</b></span>

In [ ]:
# Iterate through every node in the topology
for nodeName in topology.iterNodes():
    print(f"Saving {nodeName} SSH information...")
    tierNumber = topology.getNodeAttribute(nodeName, 'tier')
    logFile[f"tier_{tierNumber}"][nodeName]["ssh"] = manager.slice.get_node(nodeName).get_ssh_command()

Saving T-1 SSH information...
Saving L-1 SSH information...
Saving H-1 SSH information...
Saving L-2 SSH information...
Saving L-3 SSH information...
Saving L-4 SSH information...
Saving T-2 SSH information...
Saving C-1-1 SSH information...
Saving C-2-1 SSH information...
Saving C-3-1 SSH information...
Saving C-4-1 SSH information...


In [ ]:
import json
with open(f'{SLICE_NAME}_k{PORTS_PER_DEVICE}_t{NUMBER_OF_TIERS}_BGP.json', "w") as outfile:
    json.dump(logFile, outfile)

## To configure Hacker node - Working
## Run the slice cell at the top

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
    fablib.show_config()

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
from FabUtils import FabOrchestrator

# Make sure you have the manager imported.
try:
    manager = FabOrchestrator(SLICE_NAME)
except Exception as e:
    print(f"Exception: {e}")

# Upload the BGP scripts folder
manager.uploadDirectoryParallel("/home/fabric/work/NShenoy/FABRIC-Automation-main/remote_scripts/bgp_scripts", remoteLocation="/home/debian", prefixList=SEC_NODE_PREFIX)

manager.executeCommandsParallel('sudo chmod +x /home/debian/bgp_scripts/*.sh', prefixList=SEC_NODE_PREFIX)
# Run it
coreNodeConfig = "./bgp_scripts/init_bgp_Hacker.sh"
manager.executeCommandsParallel(coreNodeConfig, prefixList=SEC_NODE_PREFIX)